In [34]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, List

In [35]:
BASE_PATH = Path().resolve().parents[2]
DATASETS_PATH = BASE_PATH / "datalake" / "data-for-model"
TRAINING_DATA_FILE = DATASETS_PATH / "train" / "train_sleep_cassette.parquet" 
VALIDATION_DATA_FILE = DATASETS_PATH / "val" / "val_sleep_cassette.parquet" 
TEST_DATA_FILE = DATASETS_PATH / "test" / "test_sleep_cassette.parquet" 

In [36]:
df_train = pd.read_parquet(TRAINING_DATA_FILE, engine="fastparquet")
df_val = pd.read_parquet(VALIDATION_DATA_FILE, engine="fastparquet")
df_test = pd.read_parquet(TEST_DATA_FILE, engine="fastparquet")